In [1]:
#量化是AI软件技术栈的关键部分，用于压缩LLM或其他模型


# L2-B - Linear Quantization I: Get the Scale and Zero Point

In this lesson, continue to learn about fundamentals of linear quantization, and implement your own Linear Quantizer.

In [10]:
import torch

from helper import linear_q_with_scale_and_zero_point, linear_dequantization, plot_quantization_errors



ImportError: cannot import name 'linear_q_with_scale_and_zero_point' from 'helper' (/home/zeno/miniconda3/envs/opensora/lib/python3.10/site-packages/helper/__init__.py)

In [11]:
### a dummy tensor to test the implementation
test_tensor=torch.tensor(
    [[191.6, -13.5, 728.6],
     [92.14, 295.5,  -184],
     [0,     684.6, 245.5]]
)

In [12]:
q_min = torch.iinfo(torch.int8).min
q_max = torch.iinfo(torch.int8).max

In [13]:
q_min

-128

In [14]:
q_max

127

In [15]:
# r_min = test_tensor.min()
r_min = test_tensor.min().item()